# 2. Uproot

<br><br><br><br><br>

## What a ~complete analysis looks like in Uproot/Awkward Array

Instead of starting with small steps, let's look at where this is going, what a sample analysis looks like with these tools.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak

import uproot
import hist

In [ ]:
upfile = uproot.open("root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")
uptree = upfile["Events"]
uptree.show()

The general strategy is to get arrays in one function call (usually slow, has to read) and use them interactively afterward.

In [ ]:
muons = uptree.arrays(["Muon_pt", "Muon_eta", "Muon_phi", "Muon_charge"], cut="nMuon >= 2", how="zip", entry_stop=100000)

We've already applied an `nMuon >= 2` cut, but we can define additional cuts.

In [ ]:
os_cut = muons[:, "Muon", "charge", 0] != muons[:, "Muon", "charge", 1]
os_cut

Slicing (to be described in more detail later) can filter and reduce the structure of an array.

In [ ]:
mu1 = muons[os_cut, 0, "Muon"]
mu2 = muons[os_cut, 1, "Muon"]
mu1, mu2

Make a histogram and fill it with a calculation from the array. The mini-plot is just the way this histogram type is visualized in Jupyter.

In [ ]:
h1 = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()

In [ ]:
h1.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

Plot it using Matplotlib (for logscale).

In [ ]:
h1.plot()
plt.yscale("log")

<br><br><br><br><br>

## What a the same analysis looks like in PyROOT

In [ ]:
import ROOT
c1 = ROOT.TCanvas()

In [ ]:
rootfile = ROOT.TFile.Open("root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")
roottree = rootfile.Get("Events")

ROOT analyses (before RDataFrame; see below) are based on an event loop. Reading and calculations are done in the loop.

This is not following the "one weird trick." That's why it's slow.

In [ ]:
h2 = ROOT.TH1D("h2", "mass", 120, 0, 120)

for index, event in enumerate(roottree):
    # Analyzing a subsample means breaking out of the loop early.
    if index == 100000:
        break
    # Applying cuts means if-statements.
    if event.nMuon >= 2 and event.Muon_charge[0] != event.Muon_charge[1]:
        mu1_pt = event.Muon_pt[0]
        mu2_pt = event.Muon_pt[1]
        mu1_eta = event.Muon_eta[0]
        mu2_eta = event.Muon_eta[1]
        mu1_phi = event.Muon_phi[0]
        mu2_phi = event.Muon_phi[1]
        h2.Fill(np.sqrt(2*mu1_pt*mu2_pt*(np.cosh(mu1_eta - mu2_eta) - np.cos(mu1_phi - mu2_phi))))

In [ ]:
h2.Draw()
c1.SetLogy()
c1.Draw()

<br><br><br><br><br>

## What a the same analysis looks like in old C++

By "old C++," I mean `TTree::GetEntry`. This is also a reading + calculating loop over events.

Use `ROOT.gInterpreter.Declare` to define a C++ function in Python that we can use through PyROOT.

In [ ]:
ROOT.gInterpreter.Declare('''
void compute(TH1D& h3, TTree& roottree) {
    UInt_t nMuon;
    float Muon_pt[50];
    float Muon_eta[50];
    float Muon_phi[50];
    int32_t Muon_charge[50];

    roottree.SetBranchStatus("*", 0);
    roottree.SetBranchStatus("nMuon", 1);
    roottree.SetBranchStatus("Muon_pt", 1);
    roottree.SetBranchStatus("Muon_eta", 1);
    roottree.SetBranchStatus("Muon_phi", 1);
    roottree.SetBranchStatus("Muon_charge", 1);

    roottree.SetBranchAddress("nMuon", &nMuon);
    roottree.SetBranchAddress("Muon_pt", Muon_pt);
    roottree.SetBranchAddress("Muon_eta", Muon_eta);
    roottree.SetBranchAddress("Muon_phi", Muon_phi);
    roottree.SetBranchAddress("Muon_charge", Muon_charge);
    
    for (int index = 0; index < 100000; index++) {
        roottree.GetEntry(index);
        if (nMuon >= 2 && Muon_charge[0] != Muon_charge[1]) {
            float mu1_pt = Muon_pt[0];
            float mu2_pt = Muon_pt[1];
            float mu1_eta = Muon_eta[0];
            float mu2_eta = Muon_eta[1];
            float mu1_phi = Muon_phi[0];
            float mu2_phi = Muon_phi[1];
            h3.Fill(sqrt(2*mu1_pt*mu2_pt*(cosh(mu1_eta - mu2_eta) - cos(mu1_phi - mu2_phi))));
        }
    }
}
''')

In [ ]:
rootfile = ROOT.TFile.Open("root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")
roottree = rootfile.Get("Events")

h3 = ROOT.TH1D("h3", "mass", 120, 0, 120)

ROOT.compute(h3, roottree)

In [ ]:
h3.Draw()
c1.SetLogy()
c1.Draw()

<br><br><br><br><br>

## What a the same analysis looks like in modern RDataFrame

This case mixes Python (for organization) with C++ (for speed).

<img src="img/rdataframe-flow.svg" style="width: 800px">

In [ ]:
df = ROOT.RDataFrame("Events", "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")

# Each node is connected to the previous, in a chain (which can split and recombine).
df_limit = df.Range(100000)
df_2mu = df_limit.Filter("nMuon >= 2")
df_os = df_2mu.Filter("Muon_charge[0] != Muon_charge[1]")

# This node is a big C++ block.
df_mass = df_os.Define("Dimuon_mass", '''
float mu1_pt = Muon_pt[0];
float mu2_pt = Muon_pt[1];
float mu1_eta = Muon_eta[0];
float mu2_eta = Muon_eta[1];
float mu1_phi = Muon_phi[0];
float mu2_phi = Muon_phi[1];
return sqrt(2*mu1_pt*mu2_pt*(cosh(mu1_eta - mu2_eta) - cos(mu1_phi - mu2_phi)));
''')

# This one is an endpoint (action).
h4 = df_mass.Histo1D(("h3", "mass", 120, 0, 120), "Dimuon_mass")

The above just sets up the calculation (compiling the C++ strings). It runs when you evaluate `h4.Draw`.

In [ ]:
h4.Draw()   # <--- This is the line that computes everything.
c1.SetLogy()
c1.Draw()

For more on RDataFrame, see [this tutorial](https://cms-opendata-workshop.github.io/workshop-lesson-root/05-rdataframe/index.html).

<br><br><br><br><br>

## Ways to get data from Uproot

FIXME